<a href="https://colab.research.google.com/github/EdjeElectronics/Train-and-Deploy-YOLO-Models/blob/main/Train_YOLO_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is still a work in progress! - Evan Juras 2024-12-31


# Train YOLO Models in Google Colab
**Author:** Evan Juras, [EJ Technology Consultants](https://ejtech.io)

**Last updated:** December 31, 2024

# Introduction

This notebook uses [Ultralytics](https://docs.ultralytics.com/) to train YOLO11, YOLOv8, or YOLOv5 object detection models with a custom dataset. At the end of this Colab, you'll have a custom YOLO model that you can run on your PC, phone, or edge device like the Raspberry Pi.

<p align=center>
<img src="https://s3.us-west-1.amazonaws.com/evanjuras.com/img/yolo-model-demo.gif" height="360"><br>
<i>Custom YOLO candy detection model in action!</i>
</p>

I created a YouTube video that walks through this guide step by step.I recommend following along with the video while working through this notebook.

*Youtube video will be added when it's ready!*

**Important note: This notebook will be continuously updated to make sure it works with newer versions of Ultralytics and YOLO. If you see any differences between the YouTube video and this notebook, always follow the notebook!**

### Working in Colab
Colab provides a virtual machine in your browser complete with a Linux OS, filesystem, Python environment, and best of all, a free GPU. We'll install PyTorch and Ultralytics in this environment and use it to train our model. Simply click the Play button on sections of code in this notebook to execute them on the virtual machine.

### Navigation
To navigate this notebook, use the table of contents in the left sidebar to jump from section to section.





**Verify NVIDIA GPU Availability**

Make sure you're using a GPU-equipped machine by going to "Runtime" -> "Change runtime type" in the top menu bar, and then selecting one of the GPU options in the Hardware accelerator section. Click Play on the following code block to verify that the NVIDIA GPU is present and ready for training.

In [1]:
!nvidia-smi

Tue Dec 31 19:49:54 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

#1.&nbsp;Gather and Label Training Images

Before we start training, we need to gather and label images that will be used for training the object detection model. A good starting point for a proof-of-concept model is 200 images. The training images should have random objects in the image along with the desired objects, and should have a variety of backgrounds and lighting conditions.

There are a couple options for gathering images:


*   Build a custom dataset by taking your own pictures of the objects and labeling them (this typically results in the best performance)
*   Find a pre-made dataset from sources like [Roboflow Universe](), [Kaggle](), or [Google Images V7]()


If you want to build your own dataset, there are several tools available for labeling images. One good option is [Label Studio](https://labelstud.io/?utm_source=youtube&utm_medium=video&utm_campaign=edjeelectronics), a free and open-source labeling tool that has a simple workflow while providing capabilities for more advanced features. My YouTube video that walks through this notebook (link to be added soon) shows how to label images with Label Studio.

<p align=center>
<img src="https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/doc/label-studio-example.PNG" height="380"><br>
<i>Example of a candy image labeled with Label Studio.</i>
</p>

If you used Label Studio to label and export the images, they'll be exported in a `project.zip` file that contains the following:

- An `images` folder containing the images
- A `labels` folder containing the labels in YOLO annotation format
- A `classes.txt` labelmap file that contains all the classes
- A `notes.json` file that contains info specific to Label Studio (this file can be ignored)

If you obtained your dataset from another source (like Roboflow Universe) or used another tool to label your dataset, make sure the files are organized in the same folder structure.

<p align=center>
<img src="https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/doc/zipped-data-example.png" height=""><br>
<i>Organize your data in the folders shown here. See my <a href="(https://www.dropbox.com/scl/fi/9x0wby2cbxhrwd8er65ox/YOLO_coin_data_12DEC30.zip?dl=0)">Candy Detection Dataset</a> for an example.</i>
</p>

Once you've got your dataset built, put into the file structure shown above, and zipped into `data.zip`, you're ready to move on to the next step.

# 2. Upload Image Dataset and Prepare Training Data

Next, we'll upload our dataset and prepare it for training with YOLO. We'll split the dataset into train and validation folders, and we'll automatically generate the configuration file for training the model.

### 2.1 Upload images

First, we need to upload the dataset to Colab. Here are a few options for moving the `data.zip` folder into this Colab instance.

**Option 1. Upload through Google Colab**

Upload the `data.zip` file to the Google Colab instance by clicking the "Files" icon on the left hand side of the browser, and then the "Upload to session storage" icon. Select the zip folder to upload it.

<p>
<br>
<img src="https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/doc/upload-colab-files.png" height="240">
</p>



**Option 2. Copy from Google Drive**
You can also upload your images to your personal Google Drive, mount the drive on this Colab session, and copy them over to the Colab filesystem. This option works well if you want to upload the images beforehand so you don't have to wait for them to upload each time you restart this Colab. If you have more than 50MB worth of images, I recommend using this option.

First, upload the `data.zip` file to your Google Drive, and make note of the folder you uploaded them to. Replace `MyDrive/path/to/data.zip` with the path to your zip file. (For example, I uploaded the zip file to folder called "candy-dataset1", so I would use `MyDrive/candy-dataset1/data.zip` for the path). Then, run the following block of code to mount your Google Drive to this Colab session and copy the folder to this filesystem.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/MyDrive/path/to/data.zip /content

**Option 3. Use my candy detection dataset**

If you just want to test the process on a pre-made dataset, you can use my candy image dataset, which contains 160 pictures of popular candies (Skittles, Snickers, etc). Download it by running the following code block.

In [ ]:
# To use my pre-made candy dataset instead of your own custom dataset, download it here
!wget -O /content/data.zip https://s3.us-west-1.amazonaws.com/evanjuras.com/resources/candy_data_14DEC24.zip

--2024-12-31 03:30:49--  https://s3.us-west-1.amazonaws.com/evanjuras.com/resources/candy_data_14DEC24.zip
Resolving s3.us-west-1.amazonaws.com (s3.us-west-1.amazonaws.com)... 52.219.216.0, 52.219.117.152, 52.219.121.80, ...
Connecting to s3.us-west-1.amazonaws.com (s3.us-west-1.amazonaws.com)|52.219.216.0|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 236626389 (226M) [application/zip]
Saving to: ‘/content/data.zip’

/content/data.zip   100%[===================>] 225.66M  99.4MB/s    in 2.3s    

2024-12-31 03:30:51 (99.4 MB/s) - ‘/content/data.zip’ saved [236626389/236626389]



### 2.2 Create directories and split images into train and validation folders

At this point, whether you used Option 1, 2, or 3, you should be able to click the folder icon on the left and see your `data.zip` file in the list of files. Next, we'll unzip `data.zip` and create some folders to hold the images. Run the following two code blocks to unzip the data and create folders for holding the images and labels.

In [ ]:
# Unzip images to a custom data folder
!unzip -q /content/data.zip -d /content/custom_data

In [ ]:
# Make ALL the directories!! This sets up the specific directory structure needed by YOLO for training models
# TO DO: Add file checking in case the zipped folder already has train and validation folders (i.e. if they got it from Roboflow Universe)
!mkdir -p /content/datasets/my_dataset/train/images; mkdir /content/datasets/my_dataset/train/labels
!mkdir -p /content/datasets/my_dataset/validation/images; mkdir /content/datasets/my_dataset/validation/labels

Next, we'll split the images into train and validation sets:

*   **Train**: These are the actual images used to train the model. In one epoch of training, every image in the train set is passed into the neural network. The training algorithm adjusts the network weights to fit the data in the images.


*   **Validation**: These images are used to check the model's performance at the end of each training epoch.

I wrote a Python script to randomly move 90% of images to the "train" folder and 10% to the "validation" folders. Run the following code block to execute the scrpt.

In [ ]:
# Split between train and val folders

from pathlib import Path
import random
import os
import sys

# Set train and val split amounts
train_percent = 0.9  # 90% of the files go to train
val_percent = 0.1 # 10% go to validation

# Define paths to image and annotation folders
image_path = '/content/custom_data/images'
label_path = '/content/custom_data/labels'

# In case dataset got uploaded with a root "data" folder:
if os.path.is
train_img_path = '/content/datasets/my_dataset/train/images'
train_txt_path = '/content/datasets/my_dataset/train/labels'
val_img_path = '/content/datasets/my_dataset/validation/images'
val_txt_path = '/content/datasets/my_dataset/validation/labels'

# Get list of all images and annotation files
img_file_list = [path for path in Path(image_path).rglob('*')]
txt_file_list = [path for path in Path(label_path).rglob('*')]

print(f'Number of image files: {len(img_file_list)}')
print(f'Number of annotation files: {len(txt_file_list)}')

# Determine number of files to move to each folder
file_num = len(img_file_list)
train_num = int(file_num*train_percent)
val_num = file_num - train_num
print('Images moving to train: %d' % train_num)
print('Images moving to validation: %d' % val_num)

# Select files randomly and move them to train or val folders
for i, set_num in enumerate([train_num, val_num]):
  for ii in range(set_num):
    img_path = random.choice(img_file_list)
    img_fn = img_path.name
    base_fn = img_path.stem
    txt_fn = base_fn + '.txt'
    txt_path = os.path.join(label_path,txt_fn)

    if i == 0: # Move first set of files to train folders
      new_img_path, new_txt_path = train_img_path, train_txt_path
    elif i == 1: # Move second set of files to the validation folders
      new_img_path, new_txt_path = val_img_path, val_txt_path

    os.rename(img_path, os.path.join(new_img_path,img_fn))
    if os.path.exists(txt_path): # If txt path does not exist, this is a background image, so skip txt file
      os.rename(txt_path,os.path.join(new_txt_path,txt_fn))

    img_file_list.remove(img_path)


In [ ]:
!python hey.py --datapath="/content/custom_data" --train_pct=0.8

Created folder at /content/data/train/images.
Created folder at /content/data/train/labels.
Created folder at /content/data/validation/images.
Created folder at /content/data/validation/labels.
Number of image files: 162
Number of annotation files: 162
Images moving to train: 129
Images moving to validation: 33


In [ ]:
# Create objects.names file (not sure which step this should be at)
### This creates a "objects.names" file with a list of classes the object detection model will detect.
%%bash
cat <<EOF >> /content/objects.names
penny
nickel
dime
quarter
EOF

# 3. Install Requirements (Ultralytics)

Next, we'll install the Ultralytics library in this Google Colab instance. This Python library will be used to train the YOLO model.

In [ ]:
!pip install ultralytics

# 4. Configure Training


In [ ]:
# Create the .yaml file that holds training config data
# Unfortunately, there's not an easy file to copy from. I think we should just create it with the cat command, like before. Or with Python.
# TODO: do it programmatically by storing a list of classes as a python variable or something

%%bash
cat <<EOF >> /content/data.yaml
path: /content/datasets/my_dataset
train: train/images
val: validation/images

nc: 11

names: ["MMs_peanut", "MMs_regular", "airheads", "gummy_worms", "milky_way", "nerds", "skittles", "snickers", "starburst", "three_musketeers", "twizzlers"]
EOF

# 5. Train Model

In [ ]:
!yolo detect train data=/content/data.yaml model=yolo11s.pt epochs=40 imgsz=640

#6. Test Model

In [ ]:
!yolo detect predict model=runs/detect/train/weights/best.pt source=datasets/my_dataset/validation/images save=True

In [ ]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg')[:10]:
  display(Image(filename=image_path, height=400))
  print('\n')


#7. Deploy Model

First, zip and download the trained model by running the code blocks below.

In [ ]:
# Create "my_model" folder to store model weights and train results
!mkdir /content/my_model
!cp /content/runs/detect/train/weights/best.pt /content/my_model/my_model.pt
!cp -r /content/runs/detect/train /content/my_model

# Zip into "my_model.zip"
%cd my_model
!zip /content/my_model.zip my_model.pt
!zip -r /content/my_model.zip train
%cd /content

In [ ]:
from google.colab import files

files.download('/content/my_model.zip')

## Deploy Model